In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_Jun24_ntuple/*.root")

In [3]:
def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
        try:
            return ak.to_numpy(ak.flatten(x,axis=axis)) 
        except:
            return x

In [4]:
hists = {}
hists["kaon_hist"] = hist.Hist("Sim Kaons",
                                hist.Cat("Ktype", "Type of Kaons"),
                                hist.Bin("e", r"Kaon Kinetic Energy [MeV]", 80, 0, 3000)
                            )
hists["kshort_hist"] = hist.Hist("Sim Kaons",
                                hist.Cat("Ktype", "Type of Kaons"),
                                hist.Bin("e", r"Kaon Kinetic Energy [MeV]", 80, 0, 3000)
                            )
hists["klong_hist"] = hist.Hist("Sim Kaons",
                                hist.Cat("Ktype", "Type of Kaons"),
                                hist.Bin("e", r"Kaon Kinetic Energy [MeV]", 80, 0, 3000)
                            )




In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)

    sim_kaon = ldmx_events['Sim_Kaon']
    sim_kaon_n = ldmx_events['n']['Sim_Kaon']
    sim_lamb_n = ldmx_events['n']['Sim_Lambda']
    
    masks_id = {'Kshort': 310,
                'Klong': 130,
                'Kplus': 321,
                'Kminus': -321,
               }
    masses = {'Kshort': 497.648,
              'Klong': 497.648,
              'Kplus': 493.677,
              'Kminus': 493.677,
             }
    kaon_mass = 497.648,
    
    
    
    n1_mask = (abs(flat(sim_kaon_n)) == 1)
    n2_mask = (abs(flat(sim_kaon_n)) == 2)
    
    n1_maskl = (abs(flat(sim_lamb_n)) == 1)
    n2_maskl = (abs(flat(sim_lamb_n)) == 2)
    
    for part,pmask in masks_id.items():
        part_mask = (sim_kaon.pdgID == pmask)
        hists["kaon_hist"].fill(
                                     e=flat(sim_kaon.e[part_mask & n1_mask & n1_maskl]) - masses[part],
                                     Ktype= part
                                     )
        
        
    kshort_mask = (sim_kaon[n1_mask & n1_maskl].pdgID == 310)    
    klong_mask = (sim_kaon[n1_mask & n1_maskl].pdgID == 130) 
    
    hists["kshort_hist"].fill(
                            e=flat(sim_kaon.e[n1_mask & n1_maskl][kshort_mask]) - kaon_mass,
                            Ktype= '$K^0$'
                            )
    hists["klong_hist"].fill(
                            e=flat(sim_kaon.e[n1_mask & n1_maskl][klong_mask]) - kaon_mass,
                            Ktype= '$K^0$'
                            )
        
   # print(flat(sim_kaon.e[n1_mask & n1_maskl][kshort_mask]) - kaon_mass)
    
    kaon_count  = 0
    short_count = 0
    long_count  = 0
    plus_count  = 0
    minus_count = 0
    
    for element in sim_kaon[n1_mask].pdgID:
        kaon_count = kaon_count+1
        for part in element:
            if part == 310:
                short_count = short_count+1
            if part == 130:
                long_count = long_count+1
            if part == 321:
                plus_count = plus_count+1
            if part == -321:
                minus_count = minus_count+1
                
                
    test_klong  = (sim_kaon[n2_mask].pdgID == 130) 
    test_kshort = (sim_kaon[n2_mask].pdgID == 310) 
    print('long', flat(sim_kaon.e[n2_mask][test_klong]).shape)
    print('short', flat(sim_kaon.e[n2_mask][test_kshort]).shape)
    #n=1
    #split into Klong and Kshort: sim_kaon.e[test_klong]).shape about = sim_kaon.e[test_kshort]).shape
    #when apply just n1_mask, short has way more than long
    #when apply just n1_maskl, long has slightly more than short
    #when apply n1_mask and n1_maskl, way more short than long
    
    #n=2
    #when apply just n2_mask, short has more than long
    #when apply just n2_maskl, long has more than short
    #when apply n1_mask and n1_maskl, way more long than short
    
    kaon_count_2  = 0
    short_count_2 = 0
    long_count_2  = 0
    plus_count_2  = 0
    minus_count_2 = 0            
                
    for element in sim_kaon[n2_mask & n2_maskl].pdgID:
        kaon_count_2 = kaon_count_2+1
        for part in element:
            if part == 310:
                short_count_2 = short_count_2+1
            if part == 130:
                long_count_2 = long_count_2+1
            if part == 321:
                plus_count_2 = plus_count_2+1
            if part == -321:
                minus_count_2 = minus_count_2+1            

    #print(short_count) 
    #print(long_count) 
    #print(plus_count)
    #print(minus_count) 
    #print(kaon_count)
    return short_count, long_count, plus_count, minus_count, kaon_count, short_count_2, long_count_2, plus_count_2, minus_count_2, kaon_count_2






In [6]:
nchunk = 0
total_short = 0
total_long  = 0
total_plus  = 0
total_minus = 0
total_kaon  = 0

total_short_2 = 0
total_long_2  = 0
total_plus_2  = 0
total_minus_2 = 0
total_kaon_2  = 0

for chunk in ldmx_dict:
    nchunk += 1
    print('process',nchunk)
    returna = ProcessChunk(chunk, hists)
    total_short = total_short + returna[0]
    total_long  = total_long  + returna[1]
    total_plus  = total_plus  + returna[2]
    total_minus = total_minus + returna[3]
    total_kaon  = total_kaon  + returna[4]
    
    total_short_2 = total_short_2 + returna[5]
    total_long_2  = total_long_2  + returna[6]
    total_plus_2  = total_plus_2  + returna[7]
    total_minus_2 = total_minus_2 + returna[8]
    total_kaon_2  = total_kaon_2  + returna[9]
    
print(total_long+total_short)
print(total_plus)   
print('Kshort', 100*total_short/total_kaon,'% of all Sim Kaons n=1' )
print('Klong ', 100*total_long /total_kaon,'% of all Sim Kaons n=1' )
print('Kplus ', 100*total_plus /total_kaon,'% of all Sim Kaons n=1' )
print('Kminus', 100*total_minus/total_kaon,'% of all Sim Kaons n=1' )

print('Kshort', 100*total_short_2/total_kaon_2,'% of Kaons n=1 with Lambda n=1' )
print('Klong ', 100*total_long_2 /total_kaon_2,'% of Kaons n=1 with Lambda n=1' )
print('Kplus ', 100*total_plus_2 /total_kaon_2,'% of Kaons n=1 with Lambda n=1' )
print('Kminus', 100*total_minus_2/total_kaon_2,'% of Kaons n=1 with Lambda n=1' )

process 1
long (165,)
short (156,)
process 2
long (159,)
short (194,)
process 3
long (160,)
short (203,)
process 4
long (157,)
short (209,)
process 5
long (162,)
short (181,)
process 6
long (158,)
short (203,)
process 7
long (143,)
short (202,)
process 8
long (163,)
short (199,)
process 9
long (161,)
short (194,)
process 10
long (157,)
short (183,)
process 11
long (185,)
short (201,)
process 12
long (163,)
short (191,)
process 13
long (193,)
short (188,)
process 14
long (182,)
short (192,)
process 15
long (141,)
short (199,)
process 16
long (178,)
short (194,)
process 17
long (155,)
short (186,)
process 18
long (145,)
short (175,)


KeyboardInterrupt: 

In [ ]:
print(hists["kaon_hist"])
fig, ax = plt.subplots()
hist.plot1d(hists["kaon_hist"],ax=ax,clear=False);
hist.plot1d(hists["kshort_hist"]+hists["klong_hist"],ax=ax,clear=False);
#leg = ax.legend([r'$K_L$: 20.6%',r'$K^-$: 0.0%',r'$K^+$: 35.3%', r'$K_S$: 44.1%', r'$K^0$'])
leg = ax.legend([r'$K_L$: 20.7%',r'$K^-$: 0.0%',r'$K^+$: 35.7%', r'$K_S$: 43.6%', r'$K^0$'])

#ax.set_ylim(0, 9000)
ax.set_ylim(0, 5000)
#ax.set_xlim(0, 2500)
#ax.text(1700,5500,'Case with 1 K')
ax.text(1520,3000,'Case with 1 K and 1 $\Lambda$')

